In [ ]:
import scanpy as sc
import pandas as pd
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import AxesGrid
import numpy as np
import warnings
import seaborn as sns
import scipy
import re

warnings.filterwarnings("ignore")

In [ ]:
colors = {
    "Epithelial_Secretory": "#AA9228",
    "Epithelial_Absorptive": "#E3C300",
    "Monocyte": "#C37698",
    "T-Cell": "#008E74",
    "MAIT": "#63ABB9",
    "Myeloid": "#EF9684",
    "ILC": "#A0C6D3",
    "B-Cell": "#E2CEAB",
    "DC": "#FE757D",
    "Fibroblast": "#E17300",
    "Endothelial": "#E30133",
    "NK": "#4A7B89",
    "Epithelial_Progenitor": "#F7BC00",
    "Neuron": "#2A2446",
    "Erythroid": "#A5021D",
    "Eosinophil": "#782c4e",
}

In [ ]:
adata = sc.read_h5ad("../data/adata/timecourse.h5ad")

In [ ]:
batches = {
    "day6_SI": {"x": 6200, "y": 6200},
    "day6_SI_r2": {"x": 5800, "y": 5500},
    "day8_SI_Ctrl": {"x": 2400, "y": 2400},
    "day8_SI_r2": {"x": 3200, "y": 1500},
    "day30_SI": {"x": 6400, "y": 2400},
    "day30_SI_r2": {"x": 6200, "y": 6200},
    "day90_SI": {"x": 2400, "y": 2400},
    "day90_SI_r2": {"x": 1200, "y": 6200},
}

# subset anndata
adata = adata[adata.obs.batch.isin(batches.keys())]
adata

In [ ]:
def absolute_cell_counts(adata, key):
    df = adata.obs
    absolute_cells = df.groupby(["batch", key]).size().unstack()
    absolute_cells["Day"] = [
        re.findall(r"\d+", index)[0] for index in absolute_cells.index
    ]
    return absolute_cells


def relative_cell_counts(adata, key):
    df = adata.obs
    relative_freq = df.groupby(["batch", key]).size().unstack()
    relative_freq = relative_freq.div(relative_freq.sum(axis=1), axis=0) * 100
    relative_freq["Day"] = [
        re.findall(r"\d+", index)[0] for index in relative_freq.index
    ]
    return relative_freq

In [ ]:
def barplot(df, key):
    df = pd.melt(df.reset_index(), id_vars=["batch", "Day"])
    # g = sns.catplot(
    #     df,
    #     x="Day",
    #     y="value",
    #     kind="bar",
    #     col=key,
    #     sharey=False,
    #     col_wrap=4,
    # )
    # g.map(sns.swarmplot, 'Day', 'value')
    g = sns.FacetGrid(df, col=key, col_wrap=4, sharey=False)
    g.map_dataframe(
        sns.barplot,
        x="Day",
        y="value",
        capsize=0.4,
        edgecolor="black",
        linewidth=0.5,
        errwidth=1,
        color="#3A9AB2",
    )
    g.map_dataframe(sns.swarmplot, x="Day", y="value", color="#EF5703")
    return g

In [ ]:
barplot(df=absolute_cell_counts(adata, "Class"), key="Class")

In [ ]:
g = barplot(df=relative_cell_counts(adata, "Type"), key="Type")
g.savefig("out/cell_frequencies_subtype.pdf")

In [ ]:
relative_cell_counts(adata, "Type")